In [16]:
import os
from typing import List

# Create a Azure credential object
# credential = DefaultAzureCredential()

# # Default folders
# DEFAULT_FOLDER = ".\CUResults"

# Set content understanding service settings
AISERVICE_ENDPOINT = "https://ai-nccutesthub1151219931597.services.ai.azure.com/" #os.getenv("AISERVICE_ENDPOINT")
API_KEY = "8PeIIMIe35IacmL7XuTAREN67plZbnDUgScEDJtZSxO6zQ0qXGfSJQQJ99ALAC4f1cMXJ3w3AAAAACOGpWTu" #os.getenv("AISERVICE_API_KEY")
API_VERSION = "?api-version=2024-12-01-preview" #os.getenv("CU_API_VERSION")

## Set Content Understanding management api paths
PATH_ANALYZER_MANAGEMENT_ALL = "/contentunderstanding/analyzers"
PATH_ANALYZER_MANAGEMENT = "/contentunderstanding/analyzers/{analyzerId}"
PATH_ANALYZER_MANAGEMENT_OPERATION = "/contentunderstanding/analyzers/{analyzerId}/operations/{operationId}"

## Set Content Understanding inference paths
PATH_ANALYZER_INFERENCE = "/contentunderstanding/analyzers/{analyzerId}:analyze"
PATH_ANALYZER_INFERENCE_GET_IMAGE = "/contentunderstanding/analyzers/{analyzerId}/results/{operationId}/images/{imageId}"

# helper method to create an analyzer

def listAllfilesinPath(directory) -> List[str]:
    return [join(directory, f) for f in listdir(directory) if (isfile(join(directory, f)) and not f.endswith('.json'))]

def loadAnalyzerfromFile(path, analyzer_name) -> dict:
    with open(path) as json_file:
        analyzer = json.load(json_file)
    analyzer['analyzerId'] = analyzer_name
    return analyzer
    
def create_analyzer(analyzer_config: str):
    print(f"Creating analyzer with id: {analyzer_config['analyzerId']}")
    headers = {
        'Ocp-Apim-Subscription-Key': API_KEY,
        'Content-Type': 'application/json',
        'cogsvc-videoanalysis-face-identification-enable': "true"
    }
    print(f"PUT {AISERVICE_ENDPOINT + PATH_ANALYZER_MANAGEMENT.format(analyzerId=analyzer_config['analyzerId'])}")
    
    response = requests.put(AISERVICE_ENDPOINT + PATH_ANALYZER_MANAGEMENT.format(analyzerId=analyzer_config["analyzerId"]) + API_VERSION, headers=headers, json=analyzer_config)
    if ('apim-request-id' in response.headers):
        print(f"request-id: {response.headers['apim-request-id']}")
    print(response)
    if response.status_code == 201:
        final_state = poll_for_results(response.headers['Operation-Location'], 'Succeeded', 'Failed')
    else:
        final_state = response.json()
        print(final_state)
    
# helper method to delete an analyzer
def delete_analyzer(analyzer_config: str):
    headers = {
        'Ocp-Apim-Subscription-Key': API_KEY,
        'Content-Type': 'application/json',
        'cogsvc-videoanalysis-face-identification-enable': "true"
    }
    print(f"DELETE {AISERVICE_ENDPOINT + PATH_ANALYZER_MANAGEMENT.format(analyzerId=analyzer_config['analyzerId'])}")
    
    response = requests.delete(AISERVICE_ENDPOINT + PATH_ANALYZER_MANAGEMENT.format(analyzerId=analyzer_config["analyzerId"]) + API_VERSION, headers=headers)
    if ('apim-request-id' in response.headers):
        print(f"request-id: {response.headers['apim-request-id']}")
    print(response)

# helper method to patch (update) an analyzer
def patch_analyzer(analyzer_config: str):
    headers = {
        'Ocp-Apim-Subscription-Key': API_KEY,
        'Content-Type': 'application/json',
        'cogsvc-videoanalysis-face-identification-enable': "true"
    }
    
    print(f"PATCH {AISERVICE_ENDPOINT + PATH_ANALYZER_MANAGEMENT.format(analyzerId=analyzer_config['analyzerId'])}")
    
    response = requests.patch(AISERVICE_ENDPOINT + PATH_ANALYZER_MANAGEMENT.format(analyzerId=analyzer_config["analyzerId"]) + API_VERSION, headers=headers)
    if ('apim-request-id' in response.headers):
        print(f"request-id: {response.headers['apim-request-id']}")
    print(response)
    
# helper method to list all analyzers
def list_all_analyzer():
    headers = {
        'Ocp-Apim-Subscription-Key': API_KEY
    }

    response = requests.get(AISERVICE_ENDPOINT + PATH_ANALYZER_MANAGEMENT_ALL + API_VERSION , headers=headers)
    return response.json()

# helper method to list all analyzers
def list_analyzer(analyzerId):
    headers = {
        'Ocp-Apim-Subscription-Key': API_KEY
    }

    response = requests.get(AISERVICE_ENDPOINT + PATH_ANALYZER_MANAGEMENT.format(analyzerId=analyzerId) + API_VERSION, headers=headers)
    return response.json()

# helper method to poll for inferencing results
def poll_for_results(operation_location: str, success_state: str, failed_state: str, timeout: int = 300, interval: int = 2):
    """
    Polls the operation location URL until the operation reaches a success or failure state.

    Args:
        operation_location (str): The URL to poll for the operation result.
        success_state (str): The status indicating the operation succeeded.
        failed_state (str): The status indicating the operation failed.
        timeout (int, optional): Maximum time to wait in seconds. Default is 60 seconds.
        interval (int, optional): Time between polling attempts in seconds. Default is 2 seconds.

    Returns:
        dict or None: The final JSON response if successful, None otherwise.
    """
    headers = {
        'Ocp-Apim-Subscription-Key': API_KEY,
        'cogsvc-videoanalysis-face-identification-enable': "true"
    }

    # print(f'GET {operation_location}')

    elapsed_time = 0
    while elapsed_time <= timeout:
        try:
            response = requests.get(operation_location, headers=headers)
            response.raise_for_status()
            result = response.json()
            #print(response)
            # print(result)

            status = result.get('status')
            if status == success_state:
                return result
            elif status == failed_state:
                print(f"Operation failed with status: {status}")
                return None

            time.sleep(interval)
            elapsed_time += interval

        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            return None

    print("Operation timed out.")

# delete any existing analyzers
analyzers = list_all_analyzer()
# print(analyzers['value'])
for analyzer in analyzers['value']:
    if analyzer.get('analyzerId').startswith('prebuilt'):
        continue
    print(analyzer.get('analyzerId'))
    delete_result = delete_analyzer(analyzer)
    print(delete_result)

# Load an Analyzer from a file
mmi_analyzer_file = loadAnalyzerfromFile("ckm-analyzer_config.json", "ckm-analyzer")
create_analyzer(mmi_analyzer_file)

Creating analyzer with id: ckm-analyzer
PUT https://ai-nccutesthub1151219931597.services.ai.azure.com//contentunderstanding/analyzers/ckm-analyzer
request-id: fa6ce838-4759-4838-b887-608d84c8d117
<Response [201]>


In [ ]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from azure.identity import DefaultAzureCredential
import pandas as pd
from datetime import datetime, timedelta 

# helper method to poll for inferencing results
def poll_for_results(operation_location: str, success_state: str, failed_state: str, timeout: int = 300, interval: int = 2):
    """
    Polls the operation location URL until the operation reaches a success or failure state.

    Args:
        operation_location (str): The URL to poll for the operation result.
        success_state (str): The status indicating the operation succeeded.
        failed_state (str): The status indicating the operation failed.
        timeout (int, optional): Maximum time to wait in seconds. Default is 60 seconds.
        interval (int, optional): Time between polling attempts in seconds. Default is 2 seconds.

    Returns:
        dict or None: The final JSON response if successful, None otherwise.
    """
    headers = {
        'Ocp-Apim-Subscription-Key': API_KEY,
        'cogsvc-videoanalysis-face-identification-enable': "true"
    }

    # print(f'GET {operation_location}')

# Set content understanding service settings
AISERVICE_ENDPOINT = "https://ai-nccutesthub1151219931597.services.ai.azure.com/" #os.getenv("AISERVICE_ENDPOINT")
API_KEY = "8PeIIMIe35IacmL7XuTAREN67plZbnDUgScEDJtZSxO6zQ0qXGfSJQQJ99ALAC4f1cMXJ3w3AAAAACOGpWTu" #os.getenv("AISERVICE_API_KEY")
API_VERSION = "?api-version=2024-12-01-preview" #os.getenv("CU_API_VERSION")

# Create a Azure credential object
credential = DefaultAzureCredential()

# Initialize Blob Storage
## Get storage information from environment variables
# BLOB_ENDPOINT = 'https://nc2202storageaccount.blob.core.windows.net/' #os.getenv("BLOB_ENDPOINT")
# BLOB_CONTAINER_NAME_AUDIO = 'data' #os.getenv("BLOB_CONTAINER_NAME_AUDIO")

BLOB_ENDPOINT = 'https://nckm64storageaccount.blob.core.windows.net/' #os.getenv("BLOB_ENDPOINT")
BLOB_CONTAINER_NAME_AUDIO = 'data' #os.getenv("BLOB_CONTAINER_NAME_AUDIO")

data = None
## Initialize BlobServiceClient
blob_service_client = BlobServiceClient(account_url=BLOB_ENDPOINT, credential=credential)
# results_container_client = blob_service_client.get_container_client(container=BLOB_CONTAINER_NAME)
audio_container_client = blob_service_client.get_container_client(container=BLOB_CONTAINER_NAME_AUDIO)

# Set blob folder to process
blob_folder_list = ['audiofiles']
output_folder_blob = 'cu_output'

## Set analyzer_name for testing with other previously created analyzers
analyzer_name = "ckm-analyzer"

# # Get the analyzer configuration
# analyzer_config = list_analyzer(analyzer_name)
# analyzer_id = analyzer_config["analyzerId"]
# print(f"Analyzer ID: {analyzer_id}")
# # Get timestamp and create output folder
# timestamp = time.strftime("%Y%m%d-%H%M%S")

analyzer_id = "ckm-analyzer"
PATH_ANALYZER_INFERENCE = "/contentunderstanding/analyzers/{analyzerId}:analyze"

headers = {
    'Ocp-Apim-Subscription-Key': API_KEY,
    'Content-Type': 'application/json',
    'cogsvc-videoanalysis-face-identification-enable': "true"
}

finalresult = []
blob_folder = blob_folder_list[0]
audio_data_client = ContainerClient.list_blobs(self = audio_container_client, name_starts_with=blob_folder)
results_folder_blob = 'calltranscriptsoutput'
results_container_client = blob_service_client.get_container_client(container=BLOB_CONTAINER_NAME_AUDIO)

# Process files from blob storage
for blob in audio_data_client:
    if blob.name.startswith('labelingProjects'):
        print('Skipping AI Foundry project working folder: ' + blob.name)

    if blob.name.startswith(blob_folder) and blob.name.endswith('.wav'):
        print(f"\nFile: {blob.name}")            
        blob_client = audio_container_client.get_blob_client(blob)
        blob_url = blob_client.url
        # print(f"Blob URL: {blob_url}")

        data = {
            "url": blob_url
        }

        data = json.dumps(data)

        # Construct the request URL
        url = f"{AISERVICE_ENDPOINT}{PATH_ANALYZER_INFERENCE.format(analyzerId=analyzer_id)}{API_VERSION}"
        # print(url)
        # Make the POST request
        response = requests.post(url, headers=headers, data=data)
        # print(response)
        operation_location = response.headers.get('Operation-Location')
      
        # Poll for results
        result = poll_for_results(operation_location, 'Succeeded', 'Failed')
        # print(result)
        file_name = (blob.name).split('/')[-1]
        start_time = file_name.replace(".wav", "")[-19:]
        timestamp_format = "%Y-%m-%d %H_%M_%S"  # Adjust format if necessary
        start_timestamp = datetime.strptime(start_time, timestamp_format)
        start_date = start_timestamp.strftime("%Y-%m-%d")
        conversation_id = file_name.split('convo_', 1)[1].split('_')[0]
        duration = int(result['result']['contents'][0]['fields']['Duration']['valueString'])
        duration = 100
        end_timestamp = str(start_timestamp + timedelta(milliseconds=duration))
        end_timestamp = end_timestamp.split(".")[0]
        print(result)
        # conversationRow = {
        #     "conversationId": conversation_id,
        #     "ConversationDate": start_date,
        #     "StartTime": str(start_timestamp),
        #     "Duration": duration,
        #     "EndTime": str(end_timestamp),
        #     "Content": result['result']['contents'][0]['fields']['content']['valueString'],
        #     "summary": result['result']['contents'][0]['fields']['summary']['valueString'],
        #     "satisfied": result['result']['contents'][0]['fields']['satisfied']['valueString'],
        #     "sentiment": result['result']['contents'][0]['fields']['sentiment']['valueString'],
        #     "topic": result['result']['contents'][0]['fields']['topic']['valueString'],
        #     "keyPhrases": result['result']['contents'][0]['fields']['keyPhrases']['valueString'],
        #     "complaint": result['result']['contents'][0]['fields']['complaint']['valueString']
        # }

        # filename = 'convo_' + str(conversation_id) + '_'+ str(start_time) + '.json'
        # processed_path = 'cu_processed_files/'
        # json.dump(conversationRow, open(processed_path + filename, 'w'), indent=4)

        # output_blob = f"{output_folder_blob}/{(blob.name).split('/')[-1]}.results.json"
        # print((blob.name).split('/')[-1])
        # # Write the results to blob storage as json file
        # result_data = json.dumps(jsonresult, indent=4)
        # print(result_data)
        # blob_results_client = results_container_client.get_blob_client(output_blob)
        # blob_results_client.upload_blob(result_data, overwrite=True)
        break


File: audiofiles/convo_05be369b-0a5d-4b6a-b7af-3aef1ba4e6e6_2024-12-08 22_00_00.wav
None
